Global setup

In [2]:
n=4
k=2
q=41 #Lithium AAA?  #same as Alkaline D
#q should be divisible by 8 for NTT
η=1 #secret key range, prefered 2 in Alkaline
γ=2^4 #y coeff range, power of 2, exponent about 2/3 of q?  Earlier version was *not* power of 2
R.<x> = ZZ[]

In [3]:
q

41

In [4]:
is_prime(q)

True

In [5]:
qlen = ceil(log(q,2))
qlen

6

In [6]:
τ = 1 #number of 1's in c
ln(binomial(4, τ)).n()+τ #challenge entropy, should be between n/2 and n

2.38629436111989

In [7]:
β = τ*η
β

1

Chance of $\mathbf{y}_i$-coefficient passing:

In [8]:
((2*(γ-β)-1)/(2*γ-1))^(n*k).n()

0.586530272919658

Expected number of repetitions:

In [9]:
1/(((2*(γ-β)-1)/(2*γ-1))^(n*k))^2.n()

2.90682666192675

Alice generates a signing key

In [10]:
A = matrix([[randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q), randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q)], 
            [randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q), randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q)]])
A

[18*x^3 + 4*x^2 + 21*x + 28  2*x^3 + 13*x^2 + 6*x + 31]
[ 30*x^3 + 20*x^2 + 2*x + 1      x^3 + x^2 + 39*x + 23]

In [11]:
s1 = matrix([[randrange(-η,η+1)*x^3+randrange(-η,η+1)*x^2+randrange(-η,η+1)*x+randrange(-η,η+1)], 
            [randrange(-η,η+1)*x^3+randrange(-η,η+1)*x^2+randrange(-η,η+1)*x+randrange(-η,η+1)]])
s1

[-x^3 - x^2 + 1]
[       x^3 - 1]

In [12]:
s2 = matrix([[randrange(-η,η+1)*x^3+randrange(-η,η+1)*x^2+randrange(-η,η+1)*x+randrange(-η,η+1)], 
            [randrange(-η,η+1)*x^3+randrange(-η,η+1)*x^2+randrange(-η,η+1)*x+randrange(-η,η+1)]])
s2

[     -x^2 - x]
[x^3 + x^2 + 1]

$s_1$ and $s_2$ are private

In [13]:
t = matrix(((A*s1 + s2) % (x^n+1) % q))
t

[-2*x^3 - 22*x^2 + 23*x + 16]
[  9*x^3 + 7*x^2 + 12*x - 38]

$A$ and $t$ are public

Frank tries to forge a message

In [14]:
M = 'BOBSUCKS'
M_list = [Mod(ord(ch) - ord("A") + 1,q).lift_centered() for ch in M]
M_list

[2, 15, 2, 19, -20, 3, 11, 19]

Pick random $\hat{z}_i$ and $\hat{c}$ and calculate $w$ (Is this necessary or could we go right for random $w$?)

In [21]:
z1hat = matrix([[randrange(-(γ-1), γ)*x^3+randrange(-(γ-1), γ)*x^2+randrange(-(γ-1), γ)*x+randrange(-(γ-1), γ)], 
            [randrange(-(γ-1), γ)*x^3+randrange(-(γ-1), γ)*x^2+randrange(-(γ-1), γ)*x+randrange(-(γ-1), γ)]])
z1hat

[ 15*x^3 - 7*x^2 + 6*x + 1]
[-10*x^3 - 6*x^2 + 2*x + 8]

In [22]:
z2hat = matrix([[randrange(-(γ-1), γ)*x^3+randrange(-(γ-1), γ)*x^2+randrange(-(γ-1), γ)*x+randrange(-(γ-1), γ)], 
            [randrange(-(γ-1), γ)*x^3+randrange(-(γ-1), γ)*x^2+randrange(-(γ-1), γ)*x+randrange(-(γ-1), γ)]])
z2hat

[12*x^3 - 13*x^2 - 13*x - 9]
[          6*x^3 + 7*x - 12]

In [27]:
clenbits = log(n,2)
clenbits

2

In [28]:
def sample_in_ball(digest):  # digest should be at least ****
    c = [0] * n
    k = n
    for i in range(n-τ, n):
        while int(digest[clenbits*k:clenbits*(k+1)], 2) > i:
            k = k+1
        j = int(digest[clenbits*k:clenbits*(k+1)], 2)
        c[i] = c[j]
        c[j] = (-1)^int(digest[i+τ-n], 2)
        k = k+1
    return c

In [33]:
chat = R(sample_in_ball(format(randrange(2^20),'020b'))) #20 random bits
chat

x^2

In [35]:
w = matrix(((A*z1hat+z2hat-chat*t) % (x^n+1) % q)).apply_map(lambda f: f.map_coefficients(lambda r: Mod(r,q).lift_centered() ))
w

[                -5*x^2 - 16]
[19*x^3 - 19*x^2 - 20*x + 12]

Start the "non-cryptographic hash/XOF (combined?)"

In [36]:
w_list = [c for row in w for p in row for c in p.padded_list(n)]  #flatten the matrix down to coefficients
w_list

[-16, 0, -5, 0, 12, -20, -19, 19]

In [37]:
def XOF(list1, list2, itemlen, numbits):
    out = ""
    list11 = list1
    while len(out)<numbits:
        newitem = vector(list11).dot_product(vector(list2)) % (2**itemlen) # make sure all bits of input are used
        out = out + format(newitem, f'0{itemlen}b')
        list11 = [newitem] + list11[:-1]
    return out[:numbits]

In [38]:
c_hash = XOF(M_list, w_list, 6, 20)
c_hash

'00001010000111011110'

Use XOF to sample a sparse polynomial

Generate the signature


In [77]:
c = R(sample_in_ball(c_hash))
c

1

Now try to find $z_i$ that match $w$, i.e. $A\mathbf{z}_1+\mathbf{z}_2 = A\hat{z}_1 + \hat{z}_2 +ct - \hat{c}t$

In [46]:
A.det() % (x^n+1) % q

33*x^3 + 8*x^2 + 29*x + 2

In [50]:
gcd(A.det(), x^n+1)

1

Could guess $z_1$ at random and solve for $z_2$

In [66]:
z1 = matrix([[randrange(-(γ-β-1), γ-β)*x^3+randrange(-(γ-𝛽-1), γ-β)*x^2+randrange(-(γ-𝛽-1), γ-β)*x+randrange(-(γ-𝛽-1), γ-β)], 
            [randrange(-(γ-β-1), γ-β)*x^3+randrange(-(γ-𝛽-1), γ-β)*x^2+randrange(-(γ-𝛽-1), γ-β)*x+randrange(-(γ-𝛽-1), γ-β)]])
z1

[-10*x^3 - 10*x^2 + 14*x - 14]
[     9*x^3 + 3*x^2 + 8*x + 8]

In [67]:
z2 = matrix((A*z1hat+z2hat+c*t-chat*t-A*z1) % (x^n+1) % q).apply_map(lambda f: f.map_coefficients(lambda r: Mod(r,q).lift_centered() ))
z2

[-20*x^3 + 18*x^2 - 17*x + 9]
[ -19*x^3 - 20*x^2 + 6*x - 3]

Is $\|\mathbf{z}_2
\|_\infty < \gamma-\beta$?

In [59]:
γ-β

15

It probably isn't

But if it is, $(\mathbf{z}_1, \mathbf{z}_2, c)$ is the signature

In [68]:
(z1, z2, c)

(
[-10*x^3 - 10*x^2 + 14*x - 14]  [-20*x^3 + 18*x^2 - 17*x + 9]   
[     9*x^3 + 3*x^2 + 8*x + 8], [ -19*x^3 - 20*x^2 + 6*x - 3], 1
)

Bob verifies the signature

In [69]:
wprime = matrix((A*z1+z2-c*t) % (x^n+1) % q).apply_map(lambda f: f.map_coefficients(lambda r: Mod(r,q).lift_centered() ))
wprime

[                -5*x^2 - 16]
[19*x^3 - 19*x^2 - 20*x + 12]

Is $\|\mathbf{z}_i\|_\infty < \gamma-\beta$?

In [70]:
z1

[-10*x^3 - 10*x^2 + 14*x - 14]
[     9*x^3 + 3*x^2 + 8*x + 8]

In [71]:
z2

[-20*x^3 + 18*x^2 - 17*x + 9]
[ -19*x^3 - 20*x^2 + 6*x - 3]

In [72]:
γ-β

15

Probably not

Is $c = H(M , w_1')$?

In [73]:
wprime_list = [c for row in wprime for p in row for c in p.padded_list(n)]  #flatten the matrix down to coefficients
wprime_list

[-16, 0, -5, 0, 12, -20, -19, 19]

In [74]:
cprime_hash = XOF(M_list, wprime_list, 6, 20)
cprime_hash

'00001010000111011110'

In [75]:
cprime = R(sample_in_ball(cprime_hash))
cprime

1

In [76]:
c == cprime

True

Other than that, the signature is valid